In [64]:
### Why do the weights need to add up to thr total smaple weight, can't find an example where previous is true.

In [18]:
import sqlite3

class GradationModule:
    def __init__(self, db_connection):
        self.db_connection = db_connection

    def calculate_cumulative_retain(self, weights):
        cumulative_retain = []
        total = 0
        for weight in weights:
            total += weight
            cumulative_retain.append(total)
        return cumulative_retain

    def calculate_percent_passing(self, cumulative_retain, total_sample_weight):
        return [
            round((total_sample_weight - retain) / total_sample_weight * 100, 2)
            for retain in cumulative_retain
        ]

    def retrieve_and_process_gradation(self, sample_id):
        try:
            cursor = self.db_connection.cursor()

            # Retrieve test information
            cursor.execute(
                "SELECT GradationTestID, TotalSampleWeight, FinesTotalWeight FROM GradationTest WHERE SampleID = ?",
                (sample_id,)
            )
            test_info = cursor.fetchone()

            if not test_info:
                return {"success": False, "message": f"No gradation test found for SampleID {sample_id}"}

            test_id = test_info[0]
            total_sample_weight = self.clean_and_convert(test_info[1])
            fines_total_weight = self.clean_and_convert(test_info[2])

            # Check for invalid TotalSampleWeight or FinesTotalWeight
            invalid_fields = []
            if total_sample_weight is None:
                invalid_fields.append("TotalSampleWeight")
            if fines_total_weight is None:
                invalid_fields.append("FinesTotalWeight")
    
            if invalid_fields:
                return {
                    "success": False,
                    "message": f"Invalid data found for {', '.join(invalid_fields)} in database. Please check SampleID {sample_id}."
                }

            # Retrieve sieve data
            cursor.execute(
                "SELECT SieveID, WeightRetained FROM GradationResults WHERE GradationTestID = ?",
                (test_id,)
            )
            sieve_data = cursor.fetchall()

            if not sieve_data:
                # Skip processing if no sieve data is found for this GradationTestID
                print(f"Warning: No sieve data found for GradationTestID {test_id}. Skipping this record.")
                return {"success": False, "message": f"No sieve data found for GradationTestID {test_id}. Skipping."}

            # Extract sieve IDs and weights
            sieve_order = [{"SieveID": sieve[0]} for sieve in sieve_data]
            weights_retained = [self.clean_and_convert(sieve[1]) for sieve in sieve_data]

            # Handle missing or invalid weight values
            if any(weight is None for weight in weights_retained):
                return {"success": False, "message": f"Invalid weight data found for GradationTestID {test_id}"}

            # Perform calculations
            cumulative_retain = self.calculate_cumulative_retain(weights_retained)
            percent_passing = self.calculate_percent_passing(cumulative_retain, total_sample_weight)

            # Store results
            self.store_gradation_results(test_id, sieve_order, weights_retained, cumulative_retain, percent_passing)
            return {"success": True, "message": f"Gradation test for SampleID {sample_id} processed successfully."}

        except sqlite3.Error as e:
            print(f"Database error: {e}")
            return {"success": False, "message": f"Database error: {e}"}
        except ValueError as e:
            print(f"Value conversion error: {e}")
            return {"success": False, "message": f"Value error: {e}"}

    def clean_and_convert(self, value):
        """
        Cleans and converts a value to float. Returns None if the value is invalid.
        """
        if value is None or value == '':
            return None
        try:
            return float(value)
        except ValueError:
            return None

    def store_gradation_results(self, test_id, sieve_order, weights_retained, cumulative_retain, percent_passing):
        cursor = self.db_connection.cursor()

        try:
            for i, sieve in enumerate(sieve_order):
                query = """
                INSERT OR REPLACE INTO GradationResults 
                (GradationTestID, SieveID, WeightRetained, CumulativeRetained, PercentPassing)
                VALUES (?, ?, ?, ?, ?);
                """
                values = (
                    test_id,
                    sieve['SieveID'],
                    weights_retained[i],
                    cumulative_retain[i],
                    percent_passing[i]
                )
                cursor.execute(query, values)

            self.db_connection.commit()
            print("Gradation results updated successfully.")
        except sqlite3.Error as e:
            print(f"Error storing gradation results: {e}")
            self.db_connection.rollback()



In [20]:
if __name__ == "__main__":
    db_path = "Soil_framework.sqlite"
    connection = sqlite3.connect(db_path)

    # Process a specific sample by its SampleID
    sample_id = 19 # Replace with the desired SampleID
    gradation_module = GradationModule(connection)
    result = gradation_module.retrieve_and_process_gradation(sample_id)
    print(result)

    connection.close()


{'success': False, 'message': 'Invalid data found for TotalSampleWeight, FinesTotalWeight in database. Please check SampleID 19.'}


In [22]:
if __name__ == "__main__":
    db_connection = sqlite3.connect('Soil_framework.sqlite')
    db_connection.row_factory = sqlite3.Row
    gradation_module = GradationModule(db_connection)

    # Fetch all test IDs from the database
    cursor = db_connection.cursor()
    cursor.execute("SELECT DISTINCT SampleID FROM GradationTest;")
    test_ids = [row['SampleID'] for row in cursor.fetchall()]

    # Process each test ID
    for test_id in test_ids:
        print(f"Processing Test ID: {test_id}")
        results = gradation_module.retrieve_and_process_gradation(test_id)

Processing Test ID: SampleID
Processing Test ID: 2
Gradation results updated successfully.
Processing Test ID: 3
Gradation results updated successfully.
Processing Test ID: 5
Processing Test ID: 6
Processing Test ID: 19
Processing Test ID: 31
Processing Test ID: 32
Processing Test ID: 34
Processing Test ID: 35
Processing Test ID: 37
Processing Test ID: 38
Processing Test ID: 40
Processing Test ID: 41
Processing Test ID: 42
Processing Test ID: 44
Processing Test ID: 45
Processing Test ID: 47
Processing Test ID: 48
Processing Test ID: 49
Processing Test ID: 50
Processing Test ID: 51
Processing Test ID: 66
Processing Test ID: 70
Processing Test ID: 71
Processing Test ID: 79
Processing Test ID: 80
Processing Test ID: 85
Processing Test ID: 86
Processing Test ID: 87
Processing Test ID: 88
Processing Test ID: 90
Processing Test ID: 91
Processing Test ID: 92
Processing Test ID: 93
Processing Test ID: 94
Processing Test ID: 95
Processing Test ID: 96
Processing Test ID: 97
Processing Test ID: 9